# TDSP Stage 2 – Data Acquisition & Understanding

## Proyek: Segmentasi Pelanggan Online Retail Berbasis RFM (Spark)

Stage ini berfokus pada proses **pengambilan data**, **pemahaman struktur data**, serta **eksplorasi awal** menggunakan Apache Spark.
Dataset yang digunakan berada dalam format **Parquet**, sehingga efisien untuk diproses secara terdistribusi.

## 2.1 Deskripsi Dataset

Dataset berisi **data transaksi online retail** dengan karakteristik utama:

- Setiap baris merepresentasikan **item produk** dalam suatu transaksi
- Satu transaksi dapat terdiri dari banyak item
- Pelanggan diidentifikasi menggunakan **PostCode** sebagai *proxy customer ID*

Kolom utama yang relevan untuk analisis RFM:
- `Invoice`
- `StockCode`
- `Quantity`
- `Price`
- `InvoiceDate`
- `PostCode`
- `Country`


In [22]:
# ==============================================================
# Inisialisasi Spark Session
# ==============================================================
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("RFM-Customer-Segmentation")
    .getOrCreate()
)

spark

## 2.2 Load Data dari Parquet

Data transaksi disimpan dalam format **Parquet** pada data lake / storage.
Format ini dipilih karena:
- Columnar storage (lebih cepat untuk agregasi)
- Mendukung schema evolution
- Sangat optimal untuk Spark


In [23]:
# ==============================================================
# TDSP Stage 2 – Load Data Transaksi dari CSV.GZ (OPSI A)
# Notebook location : /notebooks
# Dataset location  : /datasets
# ==============================================================

import os
from pyspark.sql import SparkSession

# --------------------------------------------------------------
# Pastikan SparkSession aktif
# --------------------------------------------------------------
spark = (
    SparkSession.builder
    .appName("RFM-Customer-Segmentation")
    .getOrCreate()
)

# --------------------------------------------------------------
# Debug: cek working directory (penting untuk path relatif)
# --------------------------------------------------------------
print("Working directory:", os.getcwd())

# --------------------------------------------------------------
# Karena notebook berada di folder /notebooks,
# maka kita naik 1 level ke project root
# --------------------------------------------------------------
DATA_PATH = "../datasets/online_retail_raw.csv.gz"

# --------------------------------------------------------------
# Validasi path sebelum load (anti PATH_NOT_FOUND)
# --------------------------------------------------------------
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Path tidak ditemukan: {DATA_PATH}")

# --------------------------------------------------------------
# Load CSV.GZ (Spark auto-handle gzip)
# --------------------------------------------------------------
df_raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .option("mode", "FAILFAST")
    .csv(DATA_PATH)
)

# --------------------------------------------------------------
# Inspect schema & sample data
# --------------------------------------------------------------
df_raw.printSchema()
df_raw.show(5, truncate=False)

Working directory: c:\Users\U1\Documents\online-retail-rfm-clustering-1\notebooks
root
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+-----------------------------------+--------+--------------+---------+----------+--------------+
|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+-----------------------------------+--------+--------------+---------+----------+--------------+
|WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850.0   |United Kingdom|
|WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850.0   |United Kingdom|
|CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850.0   |United Kingdom|
|KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |1785

## 2.3 Pemahaman Skema & Struktur Data

Langkah awal yang penting adalah memahami:
- Tipe data setiap kolom
- Apakah terdapat kolom yang tidak relevan
- Konsistensi format tanggal dan numerik

Pemahaman ini akan sangat mempengaruhi proses **feature engineering RFM** pada stage berikutnya.

In [24]:
# ==============================================================
# Statistik dasar & jumlah baris
# ==============================================================
from pyspark.sql.functions import count

df_raw.select([count(c).alias(c) for c in df_raw.columns]).show(truncate=False)

print(f"Jumlah baris data: {df_raw.count()}")

+-----------+--------+-----------+---------+----------+-------+
|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+-----------+--------+-----------+---------+----------+-------+
|540455     |541909  |541909     |541909   |406829    |541909 |
+-----------+--------+-----------+---------+----------+-------+

Jumlah baris data: 541909


## 2.4 Pemeriksaan Kualitas Data (Data Quality Check)

Beberapa aspek kualitas data yang perlu diperiksa:
- Nilai kosong (null) pada kolom penting
- Transaksi tanpa PostCode
- Transaksi dari negara di luar scope analisis

Tahap ini bersifat **diagnostik**, belum melakukan pembersihan permanen.

In [25]:
# ==============================================================
# Cek jumlah nilai null per kolom
# ==============================================================
from pyspark.sql.functions import col, sum as spark_sum

null_summary = df_raw.select([
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in df_raw.columns
])

null_summary.show(truncate=False)

+-----------+--------+-----------+---------+----------+-------+
|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+-----------+--------+-----------+---------+----------+-------+
|1454       |0       |0          |0        |135080    |0      |
+-----------+--------+-----------+---------+----------+-------+



## 2.5 Filter Awal Berdasarkan Kebutuhan Bisnis

Sesuai dengan ruang lingkup proyek:
- Hanya transaksi dengan **PostCode valid** yang dipertahankan
- Analisis difokuskan pada **pelanggan dari UK**

Filter ini bersifat **sementara**, hasil akhirnya akan digunakan pada TDSP Stage 3.

In [26]:
from pyspark.sql.functions import col

df_filtered = (
    df_raw
    .filter(col("CustomerID").isNotNull())
    .filter(col("Country") == "United Kingdom")
)

print(f"Jumlah baris setelah filter awal: {df_filtered.count()}")
df_filtered.show(5, truncate=False)


Jumlah baris setelah filter awal: 361878
+-----------------------------------+--------+--------------+---------+----------+--------------+
|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+-----------------------------------+--------+--------------+---------+----------+--------------+
|WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850.0   |United Kingdom|
|WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850.0   |United Kingdom|
|CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850.0   |United Kingdom|
|KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850.0   |United Kingdom|
|RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850.0   |United Kingdom|
+-----------------------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [27]:
# Ambil subset yang sudah difilter
df_filtered_pd = df_filtered.toPandas()

# Simpan pakai pandas (BUKAN spark) → CSV.GZ
df_filtered_pd.to_csv(
    "../datasets/online_retail_stage2.csv.gz",
    index=False,
    compression="gzip"
)

print("✅ Stage 2 saved via pandas (csv.gz)")

✅ Stage 2 saved via pandas (csv.gz)


## 2.6 Insight Awal (Initial Observations)

Beberapa insight awal yang biasanya muncul pada tahap ini:
- Distribusi jumlah item per transaksi cenderung tidak merata
- Sebagian kecil pelanggan berkontribusi pada volume transaksi yang besar
- Terdapat indikasi pelanggan organisasi (B2B) dengan quantity tinggi

Insight ini akan diperdalam melalui perhitungan **Recency, Frequency, dan Monetary** pada stage berikutnya.

## Ringkasan Stage 2

Pada TDSP Stage 2 ini, kita telah:
- Memuat data transaksi dari format Parquet menggunakan Spark
- Memahami struktur dan skema data
- Melakukan pemeriksaan kualitas data awal
- Menerapkan filter dasar sesuai kebutuhan bisnis

Tahap selanjutnya adalah **TDSP Stage 3 – Modeling**, yang mencakup:
- Feature engineering RFM
- Normalisasi data
- Clustering pelanggan berbasis Spark